In [3]:
import pandas as pd
import streamlit as st

In [4]:
def load_data():
    sheet_id = "1kajUuZwL9l1suipRNy7t2g_SGJUcDVh6MeTu66yK-Z4"
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
    # Leer directamente como DataFrame
    df = pd.read_csv(url)
    return df

# Simplemente llama a la función sin parámetros
df = load_data()


In [5]:
# Display dataframe columns as a list
print("Columns in the dataframe:")
print(list(df.columns))

Columns in the dataframe:
['Date', 'Session Title', 'Player Name', 'Split Name', 'Tags', 'Split Start Time', 'Split End Time', 'Duration', 'Distance (km)', 'Sprint Distance (m)', 'Power Plays', 'Energy (kcal)', 'Impacts', 'Hr Load', 'Time In Red Zone (min)', 'Player Load (estres mecanico)', 'Top Speed (m/s)', 'Distance Per Min (m/min)', 'Power Score (w/kg)', 'Work Ratio', 'Hr Max (bpm)', 'Distance in Speed Zone 1 (km)', 'Distance in Speed Zone 2 (km)', 'Distance in Speed Zone 3 (km)', 'Distance in Speed Zone 4 (km)', 'Distance in Speed Zone 5 (km)', 'Time in Speed Zone 1 (secs)', 'Time in Speed Zone 2 (secs)', 'Time in Speed Zone 3 (secs)', 'Time in Speed Zone 4 (secs)', 'Time in Speed Zone 5 (secs)', 'Impact Zones: 3 - 5 G (Impacts)', 'Impact Zones: 5 - 10 G (Impacts)', 'Impact Zones: 10 - 15 G (Impacts)', 'Impact Zones: 15 - 20 G (Impacts)', 'Impact Zones: > 20 G (Impacts)', 'Power Play Duration Zones: 0 - 2.5 s (Power Plays)', 'Power Play Duration Zones: 2.5 - 5 s (Power Plays)', 'P

In [1]:
df.columns()

NameError: name 'df' is not defined

In [6]:
def obtener_estadisticas_aceleraciones(sesiones_df):
    """
    Obtiene estadísticas relacionadas con las aceleraciones y desaceleraciones de los jugadores
    
    Parámetros:
    - sesiones_df: DataFrame con los datos de sesiones filtrados
    
    Retorna:
    - DataFrame con estadísticas de aceleraciones y desaceleraciones
    """
    try:
        # Columnas de aceleraciones
        columnas_aceleraciones = [
            'Accelerations Zone Count: 2 - 3 m/s/s', 
            'Accelerations Zone Count: 3 - 4 m/s/s', 
            'Accelerations Zone Count: > 4 m/s/s'
        ]
        
        # Columnas de desaceleraciones
        columnas_desaceleraciones = [
            'Deceleration Zone Count: 1 - 2 m/s/s', 
            'Deceleration Zone Count: 2 - 3 m/s/s', 
            'Deceleration Zone Count: 3 - 4 m/s/s', 
            'Deceleration Zone Count: > 4 m/s/s'
        ]
        
        # Convertir columnas a numéricas si es necesario
        for col in columnas_aceleraciones + columnas_desaceleraciones:
            if isinstance(sesiones_df[col].iloc[0], str):
                sesiones_df[col] = sesiones_df[col].str.replace(',', '.').astype(float)
            elif not pd.api.types.is_numeric_dtype(sesiones_df[col]):
                sesiones_df[col] = pd.to_numeric(sesiones_df[col], errors='coerce')
        
        # Crear columnas con la suma de aceleraciones y desaceleraciones
        sesiones_df['Total Aceleraciones'] = sesiones_df[columnas_aceleraciones].sum(axis=1)
        sesiones_df['Total Desaceleraciones'] = sesiones_df[columnas_desaceleraciones].sum(axis=1)
        
        # Agrupar por Session Title
        aceleraciones_stats = sesiones_df.groupby(['Session Title'])[
            ['Total Aceleraciones', 'Total Desaceleraciones']].agg(['mean', 'count']).reset_index()
        
        # Reorganizar columnas para facilitar su uso
        aceleraciones_stats.columns = [
            'Sesión',
            'Media Aceleraciones', 'Count Aceleraciones',
            'Media Desaceleraciones', 'Count Desaceleraciones'
        ]
        
        # Ordenar alfabéticamente por el nombre de la sesión
        aceleraciones_stats = aceleraciones_stats.sort_values('Sesión', ascending=True)
        
        return aceleraciones_stats
        
    except Exception as e:
        print(f"Error al obtener estadísticas de aceleraciones: {e}")
        return None

def graficar_aceleraciones(aceleraciones_stats, tipo_sesion, n_sesiones=10):
    """
    Genera un gráfico de barras comparando aceleraciones (azul) y desaceleraciones (rojo)
    """
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    if aceleraciones_stats is None or aceleraciones_stats.empty:
        print("No hay datos de aceleraciones para visualizar")
        return None
    
    # Limitar a las últimas n sesiones para mejor visualización
    if len(aceleraciones_stats) > n_sesiones:
        aceleraciones_stats = aceleraciones_stats.tail(n_sesiones)
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Añadir barras para aceleraciones (en azul)
    fig.add_trace(go.Bar(
        x=aceleraciones_stats['Sesión'],
        y=aceleraciones_stats['Media Aceleraciones'],
        name='Aceleraciones',
        marker_color='rgb(65, 105, 225)',  # Azul royal
        text=aceleraciones_stats['Media Aceleraciones'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Aceleraciones: %{y:.1f}<br>Conteo: %{customdata}<extra></extra>',
        customdata=aceleraciones_stats['Count Aceleraciones']
    ))
    
    # Añadir barras para desaceleraciones (en rojo)
    fig.add_trace(go.Bar(
        x=aceleraciones_stats['Sesión'],
        y=aceleraciones_stats['Media Desaceleraciones'],
        name='Desaceleraciones',
        marker_color='rgb(220, 20, 60)',  # Rojo carmesí
        text=aceleraciones_stats['Media Desaceleraciones'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Desaceleraciones: %{y:.1f}<br>Conteo: %{customdata}<extra></extra>',
        customdata=aceleraciones_stats['Count Desaceleraciones']
    ))
    
    # Actualizar diseño
    fig.update_layout(
        title={
            'text': f'Aceleraciones vs Desaceleraciones - {tipo_sesion}',
            'font': {'size': 24},
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis=dict(
            title='Sesión',
            tickangle=-30,
            tickfont=dict(size=14),
            title_font=dict(size=16)
        ),
        yaxis=dict(
            title='Cantidad promedio',
            gridcolor='rgb(220, 220, 220)'
        ),
        plot_bgcolor='rgb(250, 250, 250)',
        paper_bgcolor='rgb(250, 250, 250)',
        font=dict(color='rgb(20, 20, 20)'),
        barmode='group',  # Barras agrupadas
        bargap=0.15,
        bargroupgap=0.1,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        height=500
    )
    
    return fig



In [7]:
# Filtrar datos por tipo de sesión (por ejemplo, "Partido")
tipo_seleccionado = "Partido"  # Puedes cambiar esto según necesites
sesiones_df = df[df['Session Title'].str.contains(tipo_seleccionado, case=False, na=False)].copy()

# Obtener estadísticas de aceleraciones y desaceleraciones
aceleraciones_stats = obtener_estadisticas_aceleraciones(sesiones_df)

# Crear gráfico
fig = graficar_aceleraciones(aceleraciones_stats, tipo_seleccionado)

# Mostrar el gráfico
fig.show()

In [8]:
def obtener_estadisticas_aceleraciones(sesiones_df):
    """
    Obtiene estadísticas relacionadas con las aceleraciones y desaceleraciones de los jugadores
    
    Parámetros:
    - sesiones_df: DataFrame con los datos de sesiones filtrados
    
    Retorna:
    - DataFrame con estadísticas de aceleraciones y desaceleraciones
    """
    try:
        # Columnas de aceleraciones
        columnas_aceleraciones = [
            'Accelerations Zone Count: 2 - 3 m/s/s', 
            'Accelerations Zone Count: 3 - 4 m/s/s', 
            'Accelerations Zone Count: > 4 m/s/s'
        ]
        
        # Columnas de desaceleraciones
        columnas_desaceleraciones = [
            'Deceleration Zone Count: 1 - 2 m/s/s', 
            'Deceleration Zone Count: 2 - 3 m/s/s', 
            'Deceleration Zone Count: 3 - 4 m/s/s', 
            'Deceleration Zone Count: > 4 m/s/s'
        ]
        
        # Convertir columnas a numéricas si es necesario
        for col in columnas_aceleraciones + columnas_desaceleraciones:
            if isinstance(sesiones_df[col].iloc[0], str):
                sesiones_df[col] = sesiones_df[col].str.replace(',', '.').astype(float)
            elif not pd.api.types.is_numeric_dtype(sesiones_df[col]):
                sesiones_df[col] = pd.to_numeric(sesiones_df[col], errors='coerce')
        
        # Crear columnas con la suma de aceleraciones y desaceleraciones
        sesiones_df['Total Aceleraciones'] = sesiones_df[columnas_aceleraciones].sum(axis=1)
        sesiones_df['Total Desaceleraciones'] = sesiones_df[columnas_desaceleraciones].sum(axis=1)
        
        # Agrupar por Session Title
        aceleraciones_stats = sesiones_df.groupby(['Session Title'])[
            ['Total Aceleraciones', 'Total Desaceleraciones']].agg(['mean', 'count']).reset_index()
        
        # Reorganizar columnas para facilitar su uso
        aceleraciones_stats.columns = [
            'Sesión',
            'Media Aceleraciones', 'Count Aceleraciones',
            'Media Desaceleraciones', 'Count Desaceleraciones'
        ]
        
        # Ordenar alfabéticamente por el nombre de la sesión
        aceleraciones_stats = aceleraciones_stats.sort_values('Sesión', ascending=True)
        
        return aceleraciones_stats
        
    except Exception as e:
        print(f"Error al obtener estadísticas de aceleraciones: {e}")
        return None

def graficar_aceleraciones(aceleraciones_stats, tipo_sesion, n_sesiones=10):
    """
    Genera un gráfico de barras comparando aceleraciones (azul) y desaceleraciones (rojo)
    """
    import plotly.graph_objects as go
    
    if aceleraciones_stats is None or aceleraciones_stats.empty:
        print("No hay datos de aceleraciones para visualizar")
        return None
    
    # Limitar a las últimas n sesiones para mejor visualización
    if len(aceleraciones_stats) > n_sesiones:
        aceleraciones_stats = aceleraciones_stats.tail(n_sesiones)
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Añadir barras para aceleraciones (en azul)
    fig.add_trace(go.Bar(
        x=aceleraciones_stats['Sesión'],
        y=aceleraciones_stats['Media Aceleraciones'],
        name='Aceleraciones',
        marker_color='rgb(65, 105, 225)',  # Azul royal
        text=aceleraciones_stats['Media Aceleraciones'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Aceleraciones: %{y:.1f}<br>Conteo: %{customdata}<extra></extra>',
        customdata=aceleraciones_stats['Count Aceleraciones']
    ))
    
    # Añadir barras para desaceleraciones (en rojo)
    fig.add_trace(go.Bar(
        x=aceleraciones_stats['Sesión'],
        y=aceleraciones_stats['Media Desaceleraciones'],
        name='Desaceleraciones',
        marker_color='rgb(220, 20, 60)',  # Rojo carmesí
        text=aceleraciones_stats['Media Desaceleraciones'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Desaceleraciones: %{y:.1f}<br>Conteo: %{customdata}<extra></extra>',
        customdata=aceleraciones_stats['Count Desaceleraciones']
    ))
    
    # Actualizar diseño
    fig.update_layout(
        title={
            'text': f'Aceleraciones vs Desaceleraciones - {tipo_sesion}',
            'font': {'size': 24},
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis=dict(
            title='Sesión',
            tickangle=-30,
            tickfont=dict(size=14),
            title_font=dict(size=16)
        ),
        yaxis=dict(
            title='Cantidad promedio',
            gridcolor='rgb(220, 220, 220)'
        ),
        plot_bgcolor='rgb(250, 250, 250)',
        paper_bgcolor='rgb(250, 250, 250)',
        font=dict(color='rgb(20, 20, 20)'),
        barmode='group',  # Barras agrupadas
        bargap=0.15,
        bargroupgap=0.1,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        height=500
    )
    
    return fig

In [9]:
def visualizar_sesion(tipo_seleccionado="Partido"):
    print(f"Analizando datos de: {tipo_seleccionado}")
    
    # Filtrar datos por tipo de sesión seleccionado
    sesiones_df = df[df['Session Title'].str.contains(tipo_seleccionado, case=False, na=False)].copy()
    
    # Mostrar cuántos registros se encontraron
    print(f"Registros encontrados: {len(sesiones_df)}")
    
    if len(sesiones_df) == 0:
        print(f"No se encontraron datos para {tipo_seleccionado}")
        return
    
    # Obtener estadísticas de aceleraciones y desaceleraciones
    aceleraciones_stats = obtener_estadisticas_aceleraciones(sesiones_df)
    
    if aceleraciones_stats is None or aceleraciones_stats.empty:
        print(f"No se pudieron calcular estadísticas para {tipo_seleccionado}")
        return
        
    print(f"Sesiones analizadas: {len(aceleraciones_stats)}")
    
    # Crear gráfico
    fig = graficar_aceleraciones(aceleraciones_stats, tipo_seleccionado)
    
    # Mostrar el gráfico
    fig.show()
    
    return aceleraciones_stats, fig  # Devolver los datos y el gráfico para uso posterior

# Probar con los diferentes tipos de sesión
tipos_sesion = ["Partido", "Martes", "Jueves"]

for tipo in tipos_sesion:
    visualizar_sesion(tipo)

Analizando datos de: Partido
Registros encontrados: 276
Sesiones analizadas: 25


Analizando datos de: Martes
Registros encontrados: 60
Sesiones analizadas: 6


Analizando datos de: Jueves
Registros encontrados: 9
Sesiones analizadas: 4
